## Create figures from the code

In [1]:
import os
os.chdir('../..')

In [2]:
import glob
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift

import analysis.plotting as pl
from modules.gait_parameters import *
import modules.numpy_funcs as nf
import modules.signals as sig


In [3]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [4]:
file_name = '2014-12-08_P004_Post_000'

In [5]:
hypo_dir = os.path.join('data', 'kinect', 'processed', 'hypothesis')
best_pos_dir = os.path.join('data', 'kinect', 'best_pos')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_best_pos = pd.read_pickle(best_pos_paths[0])

In [6]:
df_head_feet = df_best_pos

In [7]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_head_feet = df_head_feet.applymap(pd.to_numeric)

# Cluster frames with mean shift to locate the walking passes
frames = df_head_feet.index
mean_shift = MeanShift(bandwidth=60).fit(nf.to_column(frames))
labels = mean_shift.labels_

# Sort labels so that the frames are in temporal order
labels = nf.map_to_whole(labels)

# DataFrames for each walking pass in a trial
pass_dfs = list(nf.group_by_label(df_head_feet, labels))

In [8]:
plt.figure()

points = np.column_stack([frames, np.ones(len(frames))])
pl.scatter_labels(points, labels)

plt.show()

In [9]:
df_pass = pass_dfs[0]

## Foot Distance

In [10]:
plt.rc('text', usetex=True)

font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 12}

plt.rc('font', **font)  # pass in the font dict as kwargs

In [11]:
frames = df_pass.index.values

foot_pos_l = np.stack(df_pass.L_FOOT)
foot_pos_r = np.stack(df_pass.R_FOOT)
norms = np.apply_along_axis(norm, 1, foot_pos_l - foot_pos_r)

signal = 1 - sig.nan_normalize(norms)
rms = sig.root_mean_square(signal)

peak_frames, _ = sw.detect_peaks(frames, signal, window_length=3, min_height=rms)

fig = plt.figure()
foot_dist = pd.Series(norms, index=frames)

pl.scatter_series(foot_dist, c='k', s=15)
plt.plot(foot_dist, c='k', linewidth=0.7)

plt.vlines(x=peak_frames, ymin=foot_dist.max(), ymax=foot_dist.min(), color='r')

plt.xlabel('Frame')
plt.ylabel('Foot Distance [cm]')

plt.show()

In [12]:
# fig.savefig('foot_dist.pdf', format='pdf', dpi=1200)

In [13]:
_, direction_pass = direction_of_pass(df_pass)

# Assign correct sides to feet
df_pass = asi.assign_sides_pass(df_pass, direction_pass)

# Ensure there are no missing frames in the walking pass
df_pass = pf.make_index_consecutive(df_pass)
df_pass = df_pass.applymap(lambda x: x if isinstance(x, np.ndarray)
                           else np.full(3, np.nan))

In [14]:
foot_series = df_pass.R_FOOT
frames_pass = df_pass.index.values

In [15]:
foot_points = np.stack(foot_series)

step_signal = lin.line_coordinate_system(np.zeros(3), direction_pass,
                                         foot_points)

is_stance = pde.detect_phases(step_signal)

## Step Signal

In [16]:
fig = plt.figure()

points = np.column_stack((frames_pass, step_signal))
pl.scatter_labels(points, ~is_stance)
plt.xlabel('Frame')
plt.ylabel('Signal')
plt.legend(['Stance', 'Swing'])

plt.show()

In [17]:
# fig.savefig('step_signal.pdf', format='pdf', dpi=1200)

In [19]:
!pwd

/home/andrew/bitbucket_repos/pose_estimation


In [22]:
!jupyter nbconvert --to script notebooks/figures/create_figures_code.ipynb

[NbConvertApp] Converting notebook notebooks/figures/create_figures_code.ipynb to script
[NbConvertApp] Writing 3669 bytes to notebooks/figures/create_figures_code.py


In [23]:
!pwd

/home/andrew/bitbucket_repos/pose_estimation
